# Repeated Holdout Test

**Objectives**
- Load the Model(s)
- Run predictions on the model using the Training/Validation & Holdout/Test* data split using Stratified K-Folds CV (params: k=5 & random_seed=1)
- Record the performance of the models (e.g., accuracy, precision, recall, ROC-AUC, etc.) on every CV instance for use in analysis to determine possible occurence of overfitting.

*The addition of the use of Holdout/Test split here acts as an addition to the original plan of only using Training/Validation data splits.*

In [ ]:
import pandas as pd

In [ ]:
DF_LGBM_TB = pd.read_csv('../Dataset/TB/LGBM_TB.csv', low_memory=False) #<== Point these to the appropriate Training+Validation datasets.
DF_LGBM_IB = pd.read_csv('../Dataset/IB/LGBM_IB.csv', low_memory=False)

display(DF_LGBM_TB.head())
display(DF_LGBM_IB.head())

DF_CATB_TB = pd.read_csv('../Dataset/TB/CATB_TB.csv', low_memory=False)
DF_CATB_IB = pd.read_csv('../Dataset/IB/CATB_IB.csv', low_memory=False)

display(DF_CATB_TB.head())
display(DF_CATB_IB.head())